In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
print("=== Example 1 — Meeting Scheduling ===")
prompt1 = "Schedule a standup tomorrow at 10am for 30 minutes."
print("\nPrompt:\n", prompt1)
resp1 = loop_agent.run(prompt1)
if resp1.get("result") == "scheduled":
    ev = resp1["event"]
    # print in the style requested
    start_dt = datetime.fromisoformat(ev["start"])
    # convert to UTC string (we represent naive datetime as UTC for demo)
    print("\nAgent Output:\n")
    print("Intent → schedule_meeting")
    print("Event created:")
    print(f"- Title: {ev['title']}")
    print(f"- Start: {start_dt.strftime('%Y-%m-%d %H:%M')} UTC")
    print(f"- Duration: { (datetime.fromisoformat(ev['end']) - start_dt).seconds//60 } minutes")
    print("Task completed.")
else:
    print("\nAgent Output:\n", resp1)


=== Example 1 — Meeting Scheduling ===

Prompt:
 Schedule a standup tomorrow at 10am for 30 minutes.


NameError: name 'loop_agent' is not defined

In [2]:
print("\n\n=== Example 2 — Create To-Do Reminder ===")
prompt3 = "Remind me to submit my monthly report by Friday."
print("\nPrompt:\n", prompt3)
resp3 = loop_agent.run(prompt3)
todo = resp3.get("todo")
due_str = todo["due"] if todo else None
print("\nAgent Output:\n")
print(f"Todo created: '{todo['text']}'")
if due_str:
    # print as YYYY-MM-DD
    print(f"Due: {due_str}")
else:
    print("Due: (none)")
print("Task logged successfully.")



=== Example 2 — Create To-Do Reminder ===

Prompt:
 Remind me to submit my monthly report by Friday.


NameError: name 'loop_agent' is not defined

In [3]:
print("\n\n=== Example 3 — Email Summary ===")
prompt2 = "Summarize my inbox."
print("\nPrompt:\n", prompt2)
resp2 = loop_agent.run(prompt2)
print("\nAgent Output:\n")
print("Inbox Summary:")
for i, item in enumerate(resp2["summary"], start=1):
    sender = "Alice" if "alice" in item["from"] else "Bob"
    action = "Schedule meeting" if "Project sync" in item["subject"] else "Add review task"
    print(f"{i}. \"{item['subject']}\" from {sender} → Action: {action}")
print("\nTask completed successfully.")



=== Example 3 — Email Summary ===

Prompt:
 Summarize my inbox.


NameError: name 'loop_agent' is not defined

In [1]:
import re
import uuid
import threading
from datetime import datetime, date, time, timedelta
from pprint import pprint

# -------------------------
# Logger (Observability)
# -------------------------
class Logger:
    def __init__(self):
        self.calls = []
        self.task_count = 0

    def log(self, component, action, payload=None):
        entry = {
            "id": str(uuid.uuid4()),
            "time": datetime.utcnow().isoformat() + "Z",
            "component": component,
            "action": action,
            "payload": payload
        }
        self.calls.append(entry)

    def inc_task(self):
        self.task_count += 1

    def summary(self):
        return {"tool_calls": len(self.calls), "task_count": self.task_count}

logger = Logger()

# -------------------------
# Mock Data Stores / Toolset
# -------------------------
CALENDAR = []   # events: id, title, start (iso), end (iso), attendees
INBOX = [
    {"id":"e1","from":"alice@example.com","subject":"Project sync","body":"Can we meet tomorrow 10-10:30?"},
    {"id":"e2","from":"bob@example.com","subject":"Weekly report","body":"Please review the attached report and send feedback."}
]
TODOS = []
SENT_EMAILS = []

# -------------------------
# Toolset: calendar, email, todos, inbox helpers
# -------------------------
class Toolset:
    @staticmethod
    def create_event(title, start_dt, duration_min=30, attendees=None):
        attendees = attendees or []
        ev = {
            "id": str(uuid.uuid4()),
            "title": title,
            "start": start_dt.isoformat(),
            "end": (start_dt + timedelta(minutes=duration_min)).isoformat(),
            "attendees": attendees
        }
        CALENDAR.append(ev)
        logger.log("Toolset", "create_event", {"title": title, "start": ev["start"], "duration": duration_min})
        return ev

    @staticmethod
    def list_events_for_day(day):
        out = []
        for ev in CALENDAR:
            st = datetime.fromisoformat(ev["start"])
            if st.date() == day:
                out.append(ev)
        logger.log("Toolset", "list_events_for_day", {"day": day.isoformat(), "count": len(out)})
        return out

    @staticmethod
    def send_email(to, subject, body):
        msg = {
            "id": str(uuid.uuid4()),
            "to": to,
            "subject": subject,
            "body": body,
            "sent_at": datetime.utcnow().isoformat() + "Z"
        }
        SENT_EMAILS.append(msg)
        logger.log("Toolset", "send_email", {"to": to, "subject": subject})
        return msg

    @staticmethod
    def add_todo(text, due=None):
        todo = {
            "id": str(uuid.uuid4()),
            "text": text,
            "due": due.isoformat() if due else None,
            "created_at": datetime.utcnow().isoformat() + "Z"
        }
        TODOS.append(todo)
        logger.log("Toolset", "add_todo", {"text": text, "due": todo["due"]})
        return todo

    @staticmethod
    def summarize_inbox():
        summ = []
        for e in INBOX:
            summ.append({"id": e["id"], "from": e["from"], "subject": e["subject"], "snippet": e["body"][:120]})
        logger.log("Toolset", "summarize_inbox", {"count": len(summ)})
        return summ

    @staticmethod
    def extract_action_items():
        verbs = ["meet","meeting","schedule","review","send","follow up","follow-up"]
        actions = []
        for e in INBOX:
            text = (e["subject"] + " " + e["body"]).lower()
            found = [v for v in verbs if v in text]
            if found:
                actions.append({"email_id": e["id"], "from": e["from"], "subject": e["subject"], "actions": found})
        logger.log("Toolset", "extract_action_items", {"count": len(actions)})
        return actions

# -------------------------
# Calendar orchestration helpers
# -------------------------
class CalendarHelper:
    @staticmethod
    def conflicts(start_dt, end_dt):
        for ev in CALENDAR:
            ev_start = datetime.fromisoformat(ev["start"])
            ev_end = datetime.fromisoformat(ev["end"])
            if not (end_dt <= ev_start or start_dt >= ev_end):
                return True
        return False

    @staticmethod
    def suggest_slot(preferred_start, duration_min=30, step_minutes=30, window_days=7):
        candidate = preferred_start
        deadline = preferred_start + timedelta(days=window_days)
        while candidate < deadline:
            candidate_end = candidate + timedelta(minutes=duration_min)
            if not CalendarHelper.conflicts(candidate, candidate_end):
                return candidate
            candidate += timedelta(minutes=step_minutes)
        return None

# -------------------------
# Parser (intent + slots)
# -------------------------
WEEKDAYS = {"monday":0,"tuesday":1,"wednesday":2,"thursday":3,"friday":4,"saturday":5,"sunday":6}

def parse_datetime(text, base_date=None):
    if base_date is None:
        base_date = datetime.utcnow().date()
    low = text.lower()
    # day
    if "tomorrow" in low:
        day = base_date + timedelta(days=1)
    else:
        # find weekday
        m = re.search(r'\b(on|by|next)?\s*(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', low)
        if m:
            wd = m.group(2)
            today_wd = base_date.weekday()
            target = WEEKDAYS[wd]
            days_ahead = (target - today_wd) % 7
            if days_ahead == 0 and 'next' in (m.group(1) or ''):
                days_ahead = 7
            day = base_date + timedelta(days=days_ahead)
        else:
            day = base_date
    # time
    tm = re.search(r'(\b\d{1,2})(?::(\d{2}))?\s*(am|pm)?', text, flags=re.I)
    hour = 10; minute = 0
    if tm:
        hour = int(tm.group(1))
        minute = int(tm.group(2)) if tm.group(2) else 0
        ampm = (tm.group(3) or "").lower()
        if ampm == "pm" and hour < 12:
            hour += 12
        if ampm == "am" and hour == 12:
            hour = 0
    return datetime.combine(day, time(hour=hour, minute=minute))

def parse(text, base_date=None):
    low = text.lower()
    if "schedule" in low or "standup" in low or "meeting" in low:
        dt = parse_datetime(text, base_date=base_date)
        duration = 30
        if "60" in low or "1 hour" in low:
            duration = 60
        title = "Standup Meeting" if "standup" in low else "Meeting"
        attendees = []
        if "marketing" in low:
            attendees = ["marketing@company.example"]
        return {"intent":"schedule_meeting","slots":{"title":title,"start":dt,"duration":duration,"attendees":attendees}}
    if "summarize" in low or "inbox" in low:
        return {"intent":"summarize_inbox","slots":{}}
    if "remind" in low or "reminder" in low or "todo" in low:
        # parse "by friday"
        m = re.search(r'by (\w+)', low)
        due = None
        if m and m.group(1) in WEEKDAYS:
            today = (base_date or datetime.utcnow().date())
            target = WEEKDAYS[m.group(1)]
            days_ahead = (target - today.weekday()) % 7
            if days_ahead == 0:
                days_ahead = 7
            due = today + timedelta(days=days_ahead)
        return {"intent":"add_todo","slots":{"text":text,"due":due}}
    return {"intent":"unknown","slots":{"text":text}}

# -------------------------
# Agents
# -------------------------
class CalendarAgent:
    """Loop-style agent: schedule with conflict resolution and suggestion."""
    def __init__(self, toolset, helper, logger, base_date=None):
        self.toolset = toolset
        self.helper = helper
        self.logger = logger
        self.base_date = base_date

    def handle(self, slots):
        title = slots["title"]
        start = slots["start"]
        dur = slots["duration"]
        attendees = slots.get("attendees", [])

        # try preferred slot
        end = start + timedelta(minutes=dur)
        if not self.helper.conflicts(start, end):
            ev = self.toolset.create_event(title, start, duration_min=dur, attendees=attendees)
            self.logger.inc_task()
            return {"status":"scheduled","event":ev}
        # conflict -> find next slot
        next_slot = self.helper.suggest_slot(start + timedelta(minutes=30), duration_min=dur)
        if next_slot:
            ev = self.toolset.create_event(title, next_slot, duration_min=dur, attendees=attendees)
            self.logger.inc_task()
            return {"status":"scheduled_alternate","event":ev,"suggested_start":next_slot.isoformat()}
        return {"status":"failed","reason":"no_available_slot"}

class CognitiveAgent:
    """Single-step cognitive agent for summarization & action extraction."""
    def __init__(self, toolset, logger):
        self.toolset = toolset
        self.logger = logger

    def handle(self):
        summary = self.toolset.summarize_inbox()
        actions = self.toolset.extract_action_items()
        self.logger.inc_task()
        return {"summary":summary,"actions":actions}

class ParallelAgent:
    """Run multiple independent tasks in parallel (e.g., send emails & create todos)."""
    def __init__(self, toolset, logger):
        self.toolset = toolset
        self.logger = logger

    def run(self, tasks):
        results = [None] * len(tasks)
        threads = []
        def worker(i, t):
            kind = t["kind"]
            if kind == "email":
                results[i] = self.toolset.send_email(t["to"], t["subject"], t["body"])
            elif kind == "todo":
                results[i] = self.toolset.add_todo(t["text"], due=t.get("due"))
            else:
                results[i] = {"error":"unknown_task"}
            self.logger.inc_task()

        for i, t in enumerate(tasks):
            th = threading.Thread(target=worker, args=(i,t))
            th.start()
            threads.append(th)
        for th in threads:
            th.join()
        return results

# -------------------------
# Coordinator: decides which agent(s) to call
# -------------------------
class Coordinator:
    def __init__(self, parser, calendar_agent, cognitive_agent, parallel_agent, logger, base_date=None):
        self.parser = parser
        self.calendar_agent = calendar_agent
        self.cognitive_agent = cognitive_agent
        self.parallel_agent = parallel_agent
        self.logger = logger
        self.base_date = base_date

    def handle_request(self, user_text):
        parsed = self.parser(user_text, base_date=self.base_date)
        self.logger.log("Coordinator", "parse", parsed)
        intent = parsed["intent"]
        slots = parsed.get("slots", {})

        # Scheduling flow
        if intent == "schedule_meeting":
            res = self.calendar_agent.handle(slots)
            return {"intent":"schedule_meeting", "result": res}

        # Summarize flow (cognitive agent)
        if intent == "summarize_inbox":
            res = self.cognitive_agent.handle()
            return {"intent":"summarize_inbox", "result": res}

        # Reminder / todo -> create todo + optional email reminder in parallel
        if intent == "add_todo":
            todo_text = slots.get("text")
            due = slots.get("due")
            tasks = [
                {"kind":"todo", "text": todo_text, "due": due},
                {"kind":"email", "to":"me@example.com", "subject":"Reminder created", "body": f"Reminder created: {todo_text}"}
            ]
            results = self.parallel_agent.run(tasks)
            return {"intent":"add_todo","result": results}

        return {"intent":"unknown","text": user_text}

# -------------------------
# Demo: run the three example prompts
# -------------------------
# Use a fixed base date so "tomorrow" resolves to 2025-11-15
demo_base_date = date(2025, 11, 14)

calendar_agent = CalendarAgent(Toolset, CalendarHelper, logger, base_date=demo_base_date)
cognitive_agent = CognitiveAgent(Toolset, logger)
parallel_agent = ParallelAgent(Toolset, logger)
coordinator = Coordinator(parse, calendar_agent, cognitive_agent, parallel_agent, logger, base_date=demo_base_date)

# Example 1 — Meeting Scheduling
prompt1 = "Schedule a standup tomorrow at 10am for 30 minutes."
print("Prompt:\n", prompt1)
out1 = coordinator.handle_request(prompt1)
if out1["intent"] == "schedule_meeting":
    res = out1["result"]
    if res["status"].startswith("scheduled"):
        ev = res["event"]
        start_dt = datetime.fromisoformat(ev["start"])
        duration = (datetime.fromisoformat(ev["end"]) - start_dt).seconds // 60
        print("\nAgent Output:\n")
        print("Intent → schedule_meeting")
        print("Event created:")
        print(f"- Title: {ev['title']}")
        print(f"- Start: {start_dt.strftime('%Y-%m-%d %H:%M')} UTC")
        print(f"- Duration: {duration} minutes")
        print("Task completed.")
    else:
        print("\nAgent Output:\n", res)

# Example 2 — Email Summary
prompt2 = "Summarize my inbox."
print("\n\nPrompt:\n", prompt2)
out2 = coordinator.handle_request(prompt2)
if out2["intent"] == "summarize_inbox":
    summary = out2["result"]["summary"]
    actions = out2["result"]["actions"]
    print("\nAgent Output:\n")
    print("Inbox Summary:")
    for i, item in enumerate(summary, start=1):
        sender = "Alice" if "alice" in item["from"] else "Bob"
        action_suggest = "Schedule meeting" if "project sync" in item["subject"].lower() else "Add review task"
        print(f"{i}. \"{item['subject']}\" from {sender} → Action: {action_suggest}")
    print("\nTask completed successfully.")

# Example 3 — Create To-Do Reminder
prompt3 = "Remind me to submit my monthly report by Friday."
print("\n\nPrompt:\n", prompt3)
out3 = coordinator.handle_request(prompt3)
if out3["intent"] == "add_todo":
    # results[0] is todo, results[1] is email send
    res0 = out3["result"][0]
    todo = res0
    due = todo.get("due")
    print("\nAgent Output:\n")
    print(f"Todo created: '{todo['text']}'")
    if due:
        print(f"Due: {due}")
    else:
        print("Due: (none)")
    print("Task logged successfully.")

# Final: show logger summary and internal states
print("\n\nLOGGER SUMMARY:")
pprint(logger.summary())

print("\nCALENDAR STATE:")
pprint(CALENDAR)

print("\nSENT EMAILS:")
pprint(SENT_EMAILS)

print("\nTODOS:")
pprint(TODOS)


Prompt:
 Schedule a standup tomorrow at 10am for 30 minutes.

Agent Output:

Intent → schedule_meeting
Event created:
- Title: Standup Meeting
- Start: 2025-11-15 10:00 UTC
- Duration: 30 minutes
Task completed.


Prompt:
 Summarize my inbox.

Agent Output:

Inbox Summary:
1. "Project sync" from Alice → Action: Schedule meeting
2. "Weekly report" from Bob → Action: Add review task

Task completed successfully.


Prompt:
 Remind me to submit my monthly report by Friday.

Agent Output:

Todo created: 'Remind me to submit my monthly report by Friday.'
Due: 2025-11-21
Task logged successfully.


LOGGER SUMMARY:
{'task_count': 4, 'tool_calls': 8}

CALENDAR STATE:
[{'attendees': [],
  'end': '2025-11-15T10:30:00',
  'id': '9f6786e3-4dd5-49b7-bfe0-03e7e70d9faa',
  'start': '2025-11-15T10:00:00',
  'title': 'Standup Meeting'}]

SENT EMAILS:
[{'body': 'Reminder created: Remind me to submit my monthly report by Friday.',
  'id': '1568c5fb-fe1e-4446-b636-e369a9a72cfa',
  'sent_at': '2025-11-15T18